In [ ]:
# Install Library
!pip install accelerate -U
!pip install Sastrawi
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Import Library
import accelerate
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset,TensorDataset
from sklearn.preprocessing import MultiLabelBinarizer
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, AdamW
from nltk.probability import FreqDist
from nltk import ngrams
from nltk.translate.bleu_score import corpus_bleu
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import warnings

In [ ]:
# Import Dataset
df1_raw = pd.read_csv("data_1C.csv")
df1_raw.head()

,Unnamed: 0,index,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,0,13012,Subhan?Allah Kenangan yg Tidak akan Terlupakan...,1,0,1,0,0,0,0,0,1
1,1,5594,"Direksi Pertamina Dirombak, ESDM Mau Premium t...",0,0,0,0,0,0,0,0,0
2,2,347,"3. Karena berbasis teknologi digital, Presiden...",0,0,0,0,0,0,0,0,0
3,3,4356,RT USER: USER USER USER kebanyakan alasan...\n...,1,1,1,0,0,0,0,1,0
4,4,10958,Ganyang PKI dg segala anteknya... URL,1,0,0,1,0,1,0,0,0


In [ ]:
# Data Cleaning
df1 = df1_raw.drop(columns=['Unnamed: 0', 'index'])
df1.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,Subhan?Allah Kenangan yg Tidak akan Terlupakan...,1,0,1,0,0,0,0,0,1
1,"Direksi Pertamina Dirombak, ESDM Mau Premium t...",0,0,0,0,0,0,0,0,0
2,"3. Karena berbasis teknologi digital, Presiden...",0,0,0,0,0,0,0,0,0
3,RT USER: USER USER USER kebanyakan alasan...\n...,1,1,1,0,0,0,0,1,0
4,Ganyang PKI dg segala anteknya... URL,1,0,0,1,0,1,0,0,0


In [ ]:
# Preprocess Text
def preprocess_text(text):
  text = text.lower()
  text = re.sub(r'www\S+|http\S+|https\S+', '', text, flags=re.MULTILINE)
  text = re.sub(r'\@\w+|\#', '', text)
  text = re.sub(r'[^A-Za-z\s]', '', text)
  stop_words = set(stopwords.words('indonesian'))
  words = text.split()
  filtered_words = [word for word in words if word.lower() not in stop_words]
  text = ' '.join(filtered_words)
  return text
df1['Tweet'] = df1['Tweet'].apply(preprocess_text)
df1.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other
0,subhanallah kenangan yg terlupakan umat islam ...,1,0,1,0,0,0,0,0,1
1,direksi pertamina dirombak esdm premium langka,0,0,0,0,0,0,0,0,0
2,berbasis teknologi digital presiden sistem bet...,0,0,0,0,0,0,0,0,0
3,rt user user user user kebanyakan alasannsi ba...,1,1,1,0,0,0,0,1,0
4,ganyang pki dg anteknya url,1,0,0,1,0,1,0,0,0


In [ ]:
# Feature selection
X = df1['Tweet'].values
y = df1[['HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other']].values

In [ ]:
# Splitting Dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p2')
bertmodel = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p2', num_labels=9)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize
def tokenize_inputs(texts, tokenizer, max_length):
    tokenized = tokenizer.batch_encode_plus(
        texts.tolist(),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_token_type_ids=False,
        return_attention_mask=True,
        return_tensors='pt'
    )
    return tokenized

In [ ]:
# Encoding
max_length = 128
train_encodings = tokenize_inputs(X_train, tokenizer, max_length)
val_encodings = tokenize_inputs(X_val, tokenizer, max_length)
test_encodings = tokenize_inputs(X_test, tokenizer, max_length)

In [ ]:
# Ubah bentuk train, validation, dan test menjadi TensorDataset
train_dataset = TensorDataset(train_encodings['input_ids'],
                              train_encodings['attention_mask'],
                              torch.tensor(y_train, dtype=torch.float))

val_dataset = TensorDataset(val_encodings['input_ids'],
                            val_encodings['attention_mask'],
                            torch.tensor(y_val, dtype=torch.float))

test_dataset = TensorDataset(test_encodings['input_ids'],
                             test_encodings['attention_mask'],
                             torch.tensor(y_test, dtype=torch.float))

In [ ]:
# Buat DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Persiapan Training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bertmodel.to(device)
optimizer = torch.optim.AdamW(bertmodel.parameters(), lr=2e-5, eps = 1e-8)


In [ ]:
# Training
def train_model(model, train_loader, val_loader, optimizer, epochs=3):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f'Epoch {epoch+1}', leave=False):
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch+1}/{epochs}')
        print(f'Average Training Loss: {avg_train_loss}')

train_model(bertmodel, train_loader, val_loader, optimizer, epochs=10)

Epoch 1/10
Average Training Loss: 0.3433011783794923


Epoch 2/10
Average Training Loss: 0.23045582527464087


Epoch 3/10
Average Training Loss: 0.1563325576032653


Epoch 4/10
Average Training Loss: 0.11343984818368247


Epoch 5/10
Average Training Loss: 0.07525406091502218


Epoch 6/10
Average Training Loss: 0.05079303813928908


Epoch 7/10
Average Training Loss: 0.03894707649156


Epoch 8/10
Average Training Loss: 0.0390440853481943


Epoch 9/10
Average Training Loss: 0.02883272044586413


Epoch 10/10
Average Training Loss: 0.021281238986800115


In [ ]:
# Training
bertmodel
bertmodel.eval()
train_preds = []
train_labels = []
with torch.no_grad():
    for batch in tqdm(train_loader, desc='Train', leave=False):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = bertmodel(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.sigmoid(logits).cpu().numpy()
        train_preds.extend(preds)
        train_labels.extend(labels.cpu().numpy())

train_preds = np.array(train_preds) >= 0.5
train_labels = np.array(train_labels)

print(classification_report(train_labels, train_preds, target_names=['HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other']))


               precision    recall  f1-score   support

           HS       1.00      1.00      1.00      1116
      Abusive       0.99      1.00      1.00      1017
HS_Individual       1.00      0.99      0.99       720
     HS_Group       0.98      1.00      0.99       396
  HS_Religion       1.00      0.99      1.00       150
      HS_Race       0.99      0.99      0.99        91
  HS_Physical       1.00      0.97      0.98        66
    HS_Gender       0.99      0.99      0.99        67
     HS_Other       1.00      1.00      1.00       769

    micro avg       1.00      1.00      1.00      4392
    macro avg       0.99      0.99      0.99      4392
 weighted avg       1.00      1.00      1.00      4392
  samples avg       0.54      0.54      0.54      4392



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Validation
def evaluate_model(model, val_loader):
    model.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validation', leave=False):
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.sigmoid(logits).cpu().numpy()
            val_preds.extend(preds)
            val_labels.extend(labels.cpu().numpy())

    val_preds = np.array(val_preds) >= 0.5
    val_labels = np.array(val_labels)

    print(classification_report(val_labels, val_preds, target_names=['HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other']))

evaluate_model(bertmodel, val_loader)

               precision    recall  f1-score   support

           HS       0.78      0.85      0.81       142
      Abusive       0.83      0.87      0.85       124
HS_Individual       0.74      0.49      0.59       100
     HS_Group       0.42      0.81      0.55        42
  HS_Religion       0.61      0.48      0.54        23
      HS_Race       0.80      0.80      0.80        20
  HS_Physical       0.50      0.11      0.18         9
    HS_Gender       0.80      0.40      0.53        10
     HS_Other       0.69      0.77      0.73        86

    micro avg       0.72      0.74      0.73       556
    macro avg       0.69      0.62      0.62       556
 weighted avg       0.73      0.74      0.72       556
  samples avg       0.42      0.42      0.40       556



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Testing
def evaluate_test(model, test_loader):
    model.eval()
    test_preds = []
    test_labels = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc='Testing', leave=False):
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.sigmoid(logits).cpu().numpy()
            test_preds.extend(preds)
            test_labels.extend(labels.cpu().numpy())

    test_preds = np.array(test_preds) >= 0.5
    test_labels = np.array(test_labels)

    print(classification_report(test_labels, test_preds, target_names=['HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other']))


evaluate_test(bertmodel, test_loader)

               precision    recall  f1-score   support

           HS       0.81      0.88      0.84       144
      Abusive       0.82      0.90      0.85       125
HS_Individual       0.66      0.57      0.61        95
     HS_Group       0.49      0.65      0.56        49
  HS_Religion       0.83      0.62      0.71        16
      HS_Race       1.00      0.88      0.93         8
  HS_Physical       0.33      0.17      0.22         6
    HS_Gender       0.50      0.67      0.57         6
     HS_Other       0.77      0.84      0.80       110

    micro avg       0.74      0.79      0.76       559
    macro avg       0.69      0.69      0.68       559
 weighted avg       0.74      0.79      0.76       559
  samples avg       0.46      0.45      0.44       559



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Penjelasan Model dan Hyperparameter Tuning : <br>
Model yang digunakan adalah pretrained model, yaitu indobert p2 dan hyperparameter tuning yang digunakan adalah learning rate (lr) dan epsilon (eps)

Kesimpulan Classification Report:

Data Training : <br>
Model menunjukkan kinerja yang luar biasa dengan presisi, recall, dan f1-score hampir sempurna (1.00) untuk semua label. Hal ini menunjukkan bahwa model mampu mengklasifikasikan data training dengan sangat baik, dengan micro, macro, dan weighted average semuanya mendekati 1.00. Tingkat akurasi yang sangat tinggi ini menunjukkan bahwa model memiliki kecocokan yang sangat baik terhadap data training, meskipun juga mungkin terdapat overfitting.

Data Validation : <br>
Model menunjukkan performa yang lebih bervariasi dibandingkan data training. Beberapa label seperti HS_Individual dan HS_Physical memiliki f1-score yang lebih rendah, menunjukkan bahwa model mengalami kesulitan dalam mengklasifikasikan beberapa kategori dengan baik. Dengan weighted average f1-score sebesar 0.72, ini menunjukkan bahwa model masih cukup baik tetapi tidak sebaik pada data training, menandakan adanya generalisasi yang kurang sempurna ke data yang tidak dilatih.

Data Testing : <br>
model mempertahankan kinerja yang cukup baik dengan weighted average f1-score sebesar 0.76. Presisi dan recall untuk beberapa label tetap cukup tinggi, seperti HS dan Abusive, namun beberapa label seperti HS_Physical dan HS_Gender memiliki performa yang lebih rendah. Hal ini menunjukkan bahwa model mampu mengklasifikasikan data testing dengan cukup baik, meskipun masih terdapat ruang untuk peningkatan, terutama untuk kategori-kategori yang memiliki sedikit data.